In [ ]:
import numpy as np
from scipy import constants
import tabulate
import pandas as pd
import matplotlib.pyplot as plt
from IonChainTools import calcPositions,lengthScale,ion_position_potential
from tweezer_functions import *
from scipy.optimize import fsolve

#Constants in SI units
eps0 = constants.epsilon_0 
m = 39.9626*constants.atomic_mass
c = constants.c
e = constants.e
hbar = constants.hbar
pi = np.pi

# setting up parameters that we're not changing
NA = 0.1
tweezer_wavelength = 532e-9
omega_tweezer = 2*pi*c/tweezer_wavelength
df = pd.read_csv("S_P_only.csv",sep = ",",encoding = "UTF-8")
lambdares = np.array(df["wavelength (nm)"])*1e-9
omega_res = 2*pi*c/lambdares
linewidths = np.array(df["A_ki (s^-1)"])
lifetimes = linewidths

In [ ]:
N = 3
x = np.empty(N)
omega_rf_axial = 1 * 2 * pi * 1e6
U = 5e-3
beam_waist = 5e-6
omega_tw_radial = tweezer_optical_potential_to_trap_frequency(tweezer_wavelength,linewidths,omega_res,P0,beam_waist,m)[0]
print(omega_tw_radial)
tweezed_ion = 0
displacement = beam_waist * 0.9
ionspacing = ion_spacing(N,omega_rf_axial)

print(pot_derivative_with_tweeze(x, omega_rf_axial, omega_tw_radial, tweezed_ion, displacement))
print(ion_position_potential(x))

In [ ]:
tweeze_on = fsolve(pot_derivative_with_tweeze,ionspacing[0],args = (omega_rf_axial,omega_tw_radial,tweezed_ion,displacement))
print(tweeze_on)
print(ionspacing)
ionspacing[0] == fsolve(pot_derivative_with_tweeze,ionspacing[0],args = (omega_rf_axial,omega_tw_radial,tweezed_ion,displacement))

In [ ]:
tweeze_on[0] - ionspacing[0][tweezed_ion]

In [ ]:
P0 = 5e-3
FWHM = (4.5)*1e-6
z_pos = 0
lambda_beam = 532e-9

#making a list of r-positions to pass through the potential function.  This is in the frame of the laser beam,
#so 0 is the center of the beam
r_pos = np.arange((4.5/2)*0.85*1e-6,(4.5/2)*(0.95)*1e-6,0.001e-6)

rayleigh = rayleigh_length(FWHM,lambda_beam)
beampropo = beam_propogation(FWHM,rayleigh,z_pos)
I_beam = intensity(P0,FWHM,beampropo,r_pos)
a = potential_position_dependent(omega_res,linewidths,omega_tweezer,I_beam)

In [ ]:
#fitting the potential vector to a quadratic function
from scipy import optimize

def poly(x,a,b):
    return a+b*(x**2)
xdata = r_pos
ydata = a

popt, pcov = optimize.curve_fit(poly, xdata, ydata)

print(popt,pcov)

plt.plot(xdata,ydata,'r.')
plt.plot(xdata,poly(xdata, *popt) ,'b-',label=f'fit: a={popt[0]:.2e} + b={popt[1]:.2e} $x^{{2}}$')
plt.xlabel("r-position (m)")
plt.ylabel("potential (J)")
plt.title("Quadratic fit to Gaussian Potential")
plt.legend()
plt.show()

In [ ]:
#popt[1] should be 1/2*k, to get omega,w = sqrt(k/m)
fit = np.sqrt(2*popt[1]/m)
print(fit)
#I thought the fit from above should match the frequency found from below, since it's using the same beam params
P0 = 5e-3
huh = tweezer_optical_potential_to_trap_frequency(lambda_beam,linewidths,omega_res,P0,FWHM,m)
print(huh[0])
print(fit/huh[0])

In [ ]:
#calculating the r-squared value of the fit
alpha_x = tuple(popt)[1]
residuals = ydata- poly(xdata, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((ydata-np.mean(ydata))**2)
r_squared = 1 - (ss_res / ss_tot)
print(r_squared)